## Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto

## CAPSTONE PROJECT - PART ONE

In [266]:
from bs4 import BeautifulSoup
import requests

import pandas as pd
import numpy as np

import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans

import folium

In [267]:
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

#### Scrape the Wikipedia page and transform the data into a pandas dataframe

To scrape wikipedia page I used library BeautifulSoup

In [268]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(website_url,'lxml')
#print(soup.prettify())

In [269]:

data = []
columns = []

My_table = soup.find('table',{'class':'wikitable'})

for index, tr in enumerate(My_table.find_all('tr')):
    section = []
    for td in tr.find_all(['th','td']):
        section.append(td.text.rstrip('\n'))
    
#First row as header
    if (index == 0):
        columns = section
    else:
        data.append(section)
        
#converting to Pandas DataFrame
df = pd.DataFrame(data = data,columns = columns)

#change in header
df = df.rename(columns = {'Postal code':'PostalCode'})


df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


#### Dataframe clean-up:

In [270]:
#These two rows will be combined into one row with the neighborhoods separated with a comma 
df['Neighborhood'] = df.groupby('PostalCode')['Neighborhood'].transform(lambda neigh: ', '.join(neigh))

In [271]:
# Ignore cells with a borough that is Not assigned.
df = df[df['Borough'] != 'Not assigned']

#update index to be postcode
if(df.index.name != 'PostalCode'):
    df = df.set_index('PostalCode')
df.head()

,Borough,Neighborhood
PostalCode,,
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,Regent Park / Harbourfront
M6A,North York,Lawrence Manor / Lawrence Heights
M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


In [272]:
#drop duplicates
df = df.drop_duplicates()

#### Final dataframe for Project Part One:

In [273]:
#final dataframe head   
df.head(10)

,Borough,Neighborhood
PostalCode,,
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,Regent Park / Harbourfront
M6A,North York,Lawrence Manor / Lawrence Heights
M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government
M9A,Etobicoke,Islington Avenue
M1B,Scarborough,Malvern / Rouge
M3B,North York,Don Mills
M4B,East York,Parkview Hill / Woodbine Gardens


Used the .shape method to print the number of rows of dataframe:

In [274]:
df.shape

(98, 2)

## CAPSTONE PROJECT - PART TWO 

In [275]:
import geocoder
import json

!pip install geopy
from geopy.geocoders import Nominatim

I wasn't able to get the geographical coordinates of the neighborhoods using the Geocoder package. I used csv file that has the geographical coordinates of each postal code: http://cocl.us/Geospatial_data

In [276]:
data_url='http://cocl.us/Geospatial_data'
df_2=pd.read_csv(data_url)
df_2.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


#### Final Dataframe for Project - Part two

linked tables below:

In [277]:
#change in header
df_2=df_2.rename(columns = {'Postal Code':'PostalCode'})

#merge tables 
df_final=pd.merge(df, df_2, how='right', on = 'PostalCode')
df_final.head(12)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636
3,M6A,North York,Lawrence Manor / Lawrence Heights,43.718518,-79.464763
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,Malvern / Rouge,43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,Parkview Hill / Woodbine Gardens,43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
